In [1]:
# import libraries
from jetbot import Robot, Camera, bgr8_to_jpeg
from IPython.display import display
from uuid import uuid1
import ipywidgets.widgets as widgets
import traitlets, time, os

In [2]:
# define names
robot = Robot()
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

In [3]:
# build data directories
blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


In [4]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

# define buttons and actions
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.3)
    time.sleep(0.3)
    robot.stop()

def step_backward(change):
    robot.backward(0.3)
    time.sleep(0.3)
    robot.stop()

def step_left(change):
    robot.left(0.2)
    time.sleep(0.2)
    robot.stop()

def step_right(change):
    robot.right(0.2)
    time.sleep(0.2)
    robot.stop()

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    print("saved free")
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    print("saved blocked")
    
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

# display buttons
top_box = widgets.HBox([free_button, forward_button, blocked_button], layout=widgets.Layout(align_self='center'))
middle_box = widgets.HBox([left_button, backward_button, right_button], layout=widgets.Layout(align_self='center'))
bottom = widgets.HBox([free_count, stop_button, blocked_count], layout=widgets.Layout(align_self='center')) 


In [5]:
# display camera
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [6]:
# display gui
display(top_box)
display(middle_box)
display(bottom)

In [17]:
# zip files for external ml
#!zip -r -q dataset.zip dataset